<a href="https://colab.research.google.com/github/kartikgandhi/Cross-Project-Hard-Voting-Edited-Dataset/blob/main/Cross_Project_HV_train_phoenix%2Btest_zk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1233]:
from numpy import mean
from numpy import std

In [1234]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [1235]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [1236]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1237]:
#file to initalize the weights from
weights = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-phoenix data 4.2-4.3 - phoenix data 4.2-4.3.csv")


In [1238]:
weights['Change'] = weights['Change'].map({'yes': 1, 'no': 0})

In [1239]:
weights.shape

(1100, 8)

In [1240]:
weights.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,Change
0,5,0,16,44,0,0,3,1
1,9,0,23,309,1,70,10,1
2,31,1,17,154,1,58,4,1
3,2,0,16,15,1,33,3,0
4,27,0,36,544,0,78,23,1


In [1241]:
X = weights.drop('Change', axis=1)
y = weights['Change']

In [1242]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split the full train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=1)

In [1243]:
# get a list of base models
def get_models():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))
	return models

In [1244]:
# evaluate each base model
def evaluate_models(models, X_train, X_val, y_train, y_val):
	# fit and evaluate the models
	scores = list()
	for name, model in models:
		# fit the model
		model.fit(X_train, y_train)
		# evaluate the model
		yhat = model.predict(X_val)
		acc = accuracy_score(y_val, yhat)
		# store the performance
		scores.append(acc)
		# report model performance
	return scores

In [1245]:
# create the base models
models = get_models()
# fit and evaluate each model
scores = evaluate_models(models, X_train, X_val, y_train, y_val)
print(scores)

[0.6831955922865014, 0.6005509641873278, 0.5895316804407713]


In [1246]:
data = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-zk 8.5.0 - zk 8.5.0.csv")

In [1247]:
data['change'] = data['change'].map({'YES': 1, 'NO': 0})

In [1248]:
data.shape

(1078, 8)

In [1249]:
data.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,4,0,136,21,2,75,5,1
1,6,0,2,21,1,0,10,1
2,6,0,3,26,1,0,8,0
3,5,0,3,24,1,0,8,0
4,10,0,3,52,1,0,12,0


In [1250]:
X = data.drop('change', axis=1)
y = data['change']

In [1251]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [1252]:
# create the ensemble
ensemble = VotingClassifier(estimators=models, voting='hard', weights=scores)
# fit the ensemble on the training dataset
ensemble=ensemble.fit(X_train, y_train)
# make predictions on test set
yhat = ensemble.predict(X_test)

In [1253]:
# evaluate predictions
score = accuracy_score(y_test, yhat)
print('Weighted Avg Accuracy: %.3f' % (score*100))

Weighted Avg Accuracy: 91.204


In [1254]:
print(yhat)

[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0]


In [1255]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,yhat)
print(cm)

[[193   1]
 [ 18   4]]


In [1256]:
# predict probabilities
# from sklearn.metrics import roc_auc_score
# pred_prob = ensemble.predict_proba(X_test)
# auc_score = roc_auc_score(y_test, pred_prob[:,1])
# print("Area Under Curve=")
# print(auc_score)

In [1257]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,yhat)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.35533084663640024


In [1258]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, yhat)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5883317713214621


In [1259]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, yhat)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.425301040082678


In [1260]:
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95       194
           1       0.80      0.18      0.30        22

    accuracy                           0.91       216
   macro avg       0.86      0.59      0.62       216
weighted avg       0.90      0.91      0.89       216

